# Trajectory picture

In [1]:
import os
import ROOT

Welcome to JupyROOT 6.16/00


In [2]:
def simulation():
    parfile = "output/traj.para.root"
    outfile = "output/traj.simu.root"

    ROOT.ROOT.EnableThreadSafety()
    ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
    ROOT.FairLogger.GetLogger().SetLogScreenLevel("ERROR")

    vmcworkdir = os.environ["VMCWORKDIR"]
    os.environ["GEOMPATH"] = vmcworkdir + "/geometry"
    os.environ["CONFIG_DIR"] = vmcworkdir + "/gconfig"
    os.environ["PHYSICSLIST"] = "QGSP_INCLXX_HP"

    # Initialize Simulation
    run = ROOT.FairRunSim()
    run.SetName("TGeant4")
    run.SetStoreTraj(True)
    run.SetMaterials("media_r3b.geo")

    # Output
    output = ROOT.FairRootFileSink(outfile)
    run.SetSink(output)

    # Primary Generator
    generator = ROOT.FairPrimaryGenerator()
    box = ROOT.FairBoxGenerator(2112, 1)
    box.SetXYZ(0, 0, 0.0)
    box.SetThetaRange(0.0, 1.0)
    box.SetPhiRange(0.0, 360.0)
    box.SetEkinRange(0.6, 0.6)
    generator.AddGenerator(box)
    run.SetGenerator(generator)

    # Geometry
    cave = ROOT.R3BCave("Cave")
    cave.SetGeometryFileName("r3b_cave_vacuum.geo")
    run.AddModule(cave)

    doubleplane = 30
    neuland_position = ROOT.TGeoTranslation(0.0, 0.0, 15 * 100 + doubleplane * 10.0 / 2.0)
    neuland = ROOT.R3BNeuland(doubleplane, neuland_position)
    run.AddModule(neuland)

    # Prepare to run
    run.Init()
    ROOT.TVirtualMC.GetMC().SetRandom(ROOT.TRandom3(1337))
    ROOT.TVirtualMC.GetMC().SetMaxNStep(100000)

    # Runtime Database
    rtdb = run.GetRuntimeDb()
    parout = ROOT.FairParRootFileIo(True)
    parout.open(parfile)
    rtdb.setOutput(parout)
    rtdb.saveOutput()

    run.Run(40)

In [ ]:
simulation()

In [2]:
def digitization():
    inpfile = "output/traj.simu.root"
    parfile = "output/traj.para.root"
    outfile = "output/traj.digi.root"

    if not os.path.isfile(inpfile):
        print(f"Input {inpfile} does not exist")
        return

    ROOT.ROOT.EnableThreadSafety()
    ROOT.FairLogger.GetLogger().SetLogVerbosityLevel("LOW")
    ROOT.FairLogger.GetLogger().SetLogScreenLevel("WARNING")

    run = ROOT.FairRunAna()
    run.SetSource(ROOT.FairFileSource(inpfile))
    run.SetSink(ROOT.FairRootFileSink(outfile))

    # Connect Runtime Database
    rtdb = run.GetRuntimeDb()
    pario = ROOT.FairParRootFileIo(False)
    pario.open(parfile)
    rtdb.setFirstInput(pario)
    rtdb.setOutput(pario)
    rtdb.saveOutput()

    # Digitize data to hit level and create respective histograms
    run.AddTask(ROOT.R3BNeulandDigitizer())

    # Build clusters and create respective histograms
    run.AddTask(ROOT.R3BNeulandClusterFinder())

    # Find the actual primary interaction points and their clusters
    run.AddTask(ROOT.R3BNeulandPrimaryInteractionFinder())
    run.AddTask(ROOT.R3BNeulandPrimaryClusterFinder())

    # Create spectra
    run.AddTask(ROOT.R3BNeulandMCMon())
    run.AddTask(ROOT.R3BNeulandHitMon())
    run.AddTask(ROOT.R3BNeulandClusterMon("NeulandPrimaryClusters", "NeulandPrimaryClusterMon"))
    run.AddTask(ROOT.R3BNeulandClusterMon())

    run.Init()
    run.Run(0, 0)

In [3]:
# Note: Will kill the worker if simulation ran
digitization()


************************************************************* 
     initialisation for run id 1588059209
************************************************************* 
-I- FairRunTimeDB::InitContainer() FairGeoParSet
Container FairGeoParSet initialized from ROOT file.
-I- FairRunTimeDB::InitContainer() FairBaseParSet
Container FairBaseParSet initialized from ROOT file.

************************************************************* 
     initialisation for run id 1588059209
************************************************************* 
-I- FairRunTimeDB::InitContainer() FairGeoParSet
-I- FairRunTimeDB::InitContainer() FairBaseParSet
-I- FairRunTimeDB::InitContainer() R3BNeulandGeoPar
Container R3BNeulandGeoPar initialized from ROOT file.
-I- FairRunTimeDB::InitContainer() R3BFieldPar
[ERROR] init() R3BFieldPar not initialized
New field at 0, type -1
[WARN] FairRootManager::GetOutFile() deprecated. Use separate file to store additional data.
[WARN] FairRootManager::GetOutFile() deprecat

Info in <TGeoManager::CloseGeometry>: Geometry loaded from file...
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CountLevels>: max level = 3, max placements = 3000
Info in <TGeoManager::CloseGeometry>: 12002 nodes/ 6 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------
Error in <FairRuntimeDb::initContainers()>: Error occured during initialization
-W- FairRunAna::GetField: Unknown field type -1
Warning in <TTree::Bronch>: FairEventHeader cannot be split, resetting splitlevel to 0


In [32]:
from collections import defaultdict

# Note: Will fail if simulation *didn't* run in this worker -.-
f = ROOT.TFile.Open("output/traj.simu.root")
t = f.evt
t.AddFriend("evt", "output/traj.digi.root")

zoffset = 1500

alldata = []
allpoints = []
allhits = []
for e in t:
    data = []
    for i, track in enumerate(e.GeoTracks):
        datum = {}
        particle = track.GetParticle()

        lv = ROOT.TLorentzVector()
        particle.Momentum(lv)
        ekin = (lv.E() - particle.GetMass()) * 1000

        # print(i, particle.GetPdgCode(), ekin, track.GetNpoints())
        datum["pdg"] = particle.GetPdgCode()
        datum["name"] = track.GetName()
        datum["ekin"] = ekin

        tr = []
        for n in range(track.GetNpoints()):
            # Check bounds
            if (
                abs(track.GetPoint(n)[2] - (zoffset + 150)) > 300 / 2 * 1.1
                or abs(track.GetPoint(n)[1]) > 250 / 2 * 1.1
                or abs(track.GetPoint(n)[0]) > 250 / 2 * 1.1
            ):
                continue
            tr.append((track.GetPoint(n)[0] / 10, track.GetPoint(n)[1] / 10, (track.GetPoint(n)[2] - zoffset) / 10,))
        datum["track"] = tr
        data.append(datum)
    alldata.append(data)

    points = []
    for p in e.NeulandPoints:
        pos = p.GetPosition()
        points.append((pos.X() / 10, pos.Y() / 10, (pos.Z() - zoffset) / 10, p.GetLightYield()))
    allpoints.append(points)

    hits = []
    for h in e.NeulandHits:
        pos = h.GetPosition()
        hits.append((pos.X() / 10, pos.Y() / 10, (pos.Z() - zoffset) / 10, h.GetE()))
    allhits.append(hits)

In [118]:
import palettable

# colors = palettable.cartocolors.qualitative.Bold_10.colors
colors = palettable.colorbrewer.qualitative.Set1_9.colors
defaultcolor = colors[8]

particles = {}
for data in alldata:
    for datum in data:
        particles[datum["pdg"]] = {"name": datum["name"], "color": defaultcolor, "line": "loosely dashdotted"}

print(particles)

{2112: {'name': 'neutron', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000020040: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000010020: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 11: {'name': 'e-', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, -12: {'name': 'nu_e_bar', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 2212: {'name': 'proton', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000060120: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 22: {'name': 'gamma', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000040090: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000010030: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000030060: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000050110: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1

In [119]:
particles[11]["name"] = "$e^-$"
particles[-11]["name"] = "$e^+$"
particles[22]["name"] = "$\gamma$"
particles[-13]["name"] = "$\\mu^+$"
particles[12]["name"] = "$\\nu_e$"
particles[-12]["name"] = "$\\bar{\\nu}_e$"
particles[14]["name"] = "$\\nu_\mu$"
particles[-14]["name"] = "$\\bar{\\nu}_\mu$"
particles[211]["name"] = "$\pi^+$"
particles[-211]["name"] = "$\pi^-$"
particles[111]["name"] = "$\pi^0$"

particles[2112]["color"] = colors[0]
particles[2212]["color"] = colors[1]
particles[1000010020]["color"] = colors[1]
particles[1000010030]["color"] = colors[1]
particles[22]["color"] = colors[2]
particles[11]["color"] = colors[3]
particles[-11]["color"] = colors[3]
particles[1000020030]["color"] = colors[6]
particles[1000020040]["color"] = colors[6]
particles[1000020060]["color"] = colors[6]
particles[12]["color"] = colors[4]
particles[-12]["color"] = colors[4]
particles[-13]["color"] = colors[7]
particles[14]["color"] = colors[4]
particles[-14]["color"] = colors[4]
particles[211]["color"] = colors[5]
particles[-211]["color"] = colors[5]
particles[111]["color"] = colors[5]

particles[2112]["line"] = "densely dotted"
particles[2212]["line"] = "solid"
particles[1000010020]["line"] = "solid"
particles[1000010030]["line"] = "solid"
particles[22]["line"] = "dotted"
particles[11]["line"] = "dashdotted"
particles[-11]["line"] = "dashdotted"
particles[1000020030]["line"] = "solid"
particles[1000020040]["line"] = "solid"
particles[1000020060]["line"] = "solid"

particles[13] = {"name": "$\\mu^-$", "color": colors[7]}

print(particles)

{2112: {'name': 'neutron', 'color': [228, 26, 28], 'line': 'densely dotted'}, 1000020040: {'name': 'XXX', 'color': [166, 86, 40], 'line': 'solid'}, 1000010020: {'name': 'XXX', 'color': [55, 126, 184], 'line': 'solid'}, 11: {'name': '$e^-$', 'color': [152, 78, 163], 'line': 'dashdotted'}, -12: {'name': '$\\bar{\\nu}_e$', 'color': [255, 127, 0], 'line': 'loosely dashdotted'}, 2212: {'name': 'proton', 'color': [55, 126, 184], 'line': 'solid'}, 1000060120: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 22: {'name': '$\\gamma$', 'color': [77, 175, 74], 'line': 'dotted'}, 1000040090: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000010030: {'name': 'XXX', 'color': [55, 126, 184], 'line': 'solid'}, 1000030060: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000050110: {'name': 'XXX', 'color': [153, 153, 153], 'line': 'loosely dashdotted'}, 1000020030: {'name': 'XXX', 'color': [166, 86, 40], 'line': 'solid'}, 10000

In [123]:
import math
import matplotlib

cmap = matplotlib.cm.get_cmap("viridis_r")

for en in range(40):
    for mode in ["l", "n", "x"]:
        out = """\\documentclass{scrartcl}
        \\usepackage{xcolor, tikz}
        \\usepackage{pgfplots}
        \\pgfplotsset{compat=newest}
        \\pagestyle{empty}
        """

        cl = set([abs(datum["pdg"]) for datum in alldata[en]])
        for pdg in cl:
            out += "\\definecolor{pdg%s}{RGB}{%d,%d,%d}\n" % (str(abs(pdg)), *particles[pdg]["color"])

        out += "\\begin{document}\n"
        out += "\\begin{tikzpicture}[scale=0.4] %\columnwidth/252.0pt]\n"
        
        for i in range(6):
            out += "\\draw[step=0.5,very thin,gray] (%f,-12.499) grid (%f,5.25);\n" % (i - 0.001, i + 0.5)
        for i in range(6, 30):
            out += "\\draw[step=0.5,very thin,gray] (%f,-12.499) grid (%f,12.499);\n" % (i - 0.001, i + 0.5)
        out += "\\draw[very thin,gray] (0,-12.5) -- (30,-12.5) -- (30,12.5) -- (6,12.5);\n"

        maxe = 50 #max([hit[3] for hit in allhits[en]])
        for hit in allhits[en]:
            c = cmap(hit[3] / maxe)
            out += "\\definecolor{tempcolor}{rgb}{%f,%f,%f}" % (c[0], c[1], c[2])
            out += "\\draw[fill=tempcolor,fill opacity=1] (%f,%f) rectangle (%f,%f);\n" % (
                hit[2] - 0.25,
                hit[1] - 0.25,
                hit[2] + 0.25,
                hit[1] + 0.25,
            )

        for point in allpoints[en]:
            if mode == "l":
                out += "\\draw (%f,%f) circle (%f);\n" % (point[2], point[1], 0.1 * math.log(point[3]))
            if mode == "n":
                out += "\\draw (%f,%f) circle (%f);\n" % (point[2], point[1], 100 * point[3])

        for datum in alldata[en]:
            out += "\\draw[color=%s, very thick, %s] " % ("pdg%s" % str(abs(datum["pdg"])), particles[datum["pdg"]]["line"])
            out += " -- ".join([f"({t[2]}, {t[1]})" for t in datum["track"]])
            out += ";\n"

        legend = set([datum["pdg"] for datum in alldata[en]])
        i = 1
        for pdg in legend:
            if particles[pdg]["name"] != "XXX":
                out += "\\draw[color=%s, very thick, %s] (0.25,%d) -- (1.25,%d) node [right,black] {%s};\n" % (
                    "pdg%s" % str(abs(pdg)),
                    particles[pdg]["line"],
                    14 - 1.1 * i,
                    14 - 1.1 * i,
                    particles[pdg]["name"],
                )
                i += 1

        out += """
        \\begin{axis}[%
            at={(0.25cm,6.75cm)},
            hide axis,
            scale only axis,
            height=0pt,
            width=0pt,
            colormap={reverse viridis}{
                indices of colormap={
                \pgfplotscolormaplastindexof{viridis},...,0 of viridis}
            },
            colorbar horizontal,
            point meta min=0,
            point meta max=50,
            colorbar style={
                width=5cm,
                xtick={50, 25, 0},
            }]
            %\\addplot [draw=none] coordinates {(1,15)};
        \\end{axis}
        """        
                
        out += """
        \\end{tikzpicture}
        \\end{document}
        """

        with open(f"paper/tracks/tracks{en:02d}{mode}.tex", "w") as f:
            f.write(out)

        import subprocess

        subprocess.call(["lualatex", f"tracks{en:02d}{mode}.tex"], cwd="paper/tracks", stdout=subprocess.DEVNULL)
        subprocess.call(
            ["pdfcrop", f"tracks{en:02d}{mode}.pdf", f"tracks{en:02d}{mode}.pdf"],
            cwd="paper/tracks",
            stdout=subprocess.DEVNULL,
        )
        os.remove(f"paper/tracks/tracks{en:02d}{mode}.aux")
        os.remove(f"paper/tracks/tracks{en:02d}{mode}.log")